In [ ]:
import random

import numpy as np
import pandas as pd
import torch

try:
    import google.colab  # noqa: F401

    !pip install -q daml[torch] torchmetrics torchvision
    !export LC_ALL="en_US.UTF-8"
    !export LD_LIBRARY_PATH="/usr/lib64-nvidia"
    !export LIBRARY_PATH="/usr/local/cuda/lib64/stubs"
    !ldconfig /usr/lib64-nvidia
except Exception:
    pass

!pip install -q tabulate

import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"


from typing import Dict, Sequence, cast

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchmetrics
import torchvision.datasets as datasets
import torchvision.transforms.v2 as v2
from torch.utils.data import DataLoader, Dataset, Subset

from daml.metrics import Sufficiency

np.random.seed(0)
np.set_printoptions(formatter={"float": lambda x: f"{x:0.4f}"})
torch.manual_seed(0)
torch.set_float32_matmul_precision("high")
device = "cuda" if torch.cuda.is_available() else "cpu"
torch._dynamo.config.suppress_errors = True

random.seed(0)
torch.use_deterministic_algorithms(True)
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"

In [ ]:
# Create
# Download the mnist dataset and preview the images
to_tensor = v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])
train_ds = datasets.MNIST("./data", train=True, download=True, transform=to_tensor)
test_ds = datasets.MNIST("./data", train=False, download=True, transform=to_tensor)

In [ ]:
# Define our network architecture
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(6400, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = torch.flatten(x, 1)  # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


# Compile the model
model = torch.compile(Net().to(device))

# Type cast the model back to Net as torch.compile returns a Unknown
# Nothing internally changes from the cast; we are simply signaling the type
model = cast(Net, model)


def custom_train(model: nn.Module, dataset: Dataset, indices: Sequence[int]):
    # Defined only for this testing scenario
    criterion = torch.nn.CrossEntropyLoss().to(device)
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    epochs = 10

    # Define the dataloader for training
    dataloader = DataLoader(Subset(dataset, indices), batch_size=16)

    for epoch in range(epochs):
        for batch in dataloader:
            # Load data/images to device
            X = torch.Tensor(batch[0]).to(device)
            # Load targets/labels to device
            y = torch.Tensor(batch[1]).to(device)
            # Zero out gradients
            optimizer.zero_grad()
            # Forward propagation
            outputs = model(X)
            # Compute loss
            loss = criterion(outputs, y)
            # Back prop
            loss.backward()
            # Update weights/parameters
            optimizer.step()


def custom_eval(model: nn.Module, dataset: Dataset) -> Dict[str, float]:
    metric = torchmetrics.Accuracy(task="multiclass", num_classes=10).to(device)
    result = 0

    # Set model layers into evaluation mode
    model.eval()
    dataloader = DataLoader(dataset, batch_size=16)
    # Tell PyTorch to not track gradients, greatly speeds up processing
    with torch.no_grad():
        for batch in dataloader:
            # Load data/images to device
            X = torch.Tensor(batch[0]).to(device)
            # Load targets/labels to device
            y = torch.Tensor(batch[1]).to(device)
            preds = model(X)
            metric.update(preds, y)
        result = metric.compute().cpu()
    return {"Accuracy": result}

In [ ]:
# User-specified inputs

# What we use to curve-fit the sufficiency model
train_ds = Subset(train_ds, range(2000))
test_ds = Subset(test_ds, range(500))

# How much training data an evaluator agent has
dataset_size = 600
# The accuracy that an evaluator agent wants
desired_accuracies = np.array([0.98])

In [ ]:
# Sufficiency evaluation is done here (cache this or substitute it with your own sufficiency study)

# Instantiate sufficiency metric
suff = Sufficiency(
    model=model,
    train_ds=train_ds,
    test_ds=test_ds,
    train_fn=custom_train,
    eval_fn=custom_eval,
    runs=5,
    substeps=10,
)

# Train & test model
output = suff.evaluate(niter=50)

projection = Sufficiency.project(output, [dataset_size])


# Evaluate the learning curve to infer the needed amount of training data
samples_needed = Sufficiency.inv_project({"Accuracy": desired_accuracies}, output)

plot_output = Sufficiency.plot(output)

In [ ]:
# The values we pull out of sufficiency
# Whether the accuracy evaluated at the user's dataset size exceeds the user's desired accuracy
is_sufficient = projection["Accuracy"][-1] >= desired_accuracies[0]
# The plot showing 1) the points internally generated to do the curve-fit
# and 2) the curve fit
figs = plot_output
# The projected accuracy for the number of samples the user has
proj_accuracy = projection["Accuracy"][-1]
# The estimated number of samples needed to achieve the accuracy that the user wants
pred_nsamples = samples_needed["Accuracy"][0]

In [ ]:
# Create a dictionary that gradient will plot as a table
suff_preds = {
    "sufficient": is_sufficient,
    "Pred Performance": proj_accuracy,
    "Pred nsamples": pred_nsamples,
    "nsamples": dataset_size,
    "desired_perf": desired_accuracies[0] * 100,
}

In [ ]:
from gradient.slide_deck.shapes import Image, Placement, SubText, Table, Text, TextContent
from gradient.slide_deck.slidedeck import (
    DEFAULT_GRADIENT_PRESENTATION_TEMPLATE_PATH,
    DefaultGradientSlideLayouts,
    SlideDeck,
)


def generate_drift_report_table(suff_preds: dict) -> pd.DataFrame:
    drift_table = pd.DataFrame(
        {
            "Is sufficient?": ["Yes" if suff_preds["sufficient"] else "No"],
            # "Test statistic": [np.mean(preds["distance"]) for preds in drift_preds.values()],
            f"Predicted performance for {suff_preds['nsamples']} samples": [suff_preds["Pred Performance"]],
            f"Predicted number of samples needed to achieve performance of {suff_preds['desired_perf']}%": [
                suff_preds["Pred nsamples"]
            ],
        }
    )
    return drift_table


def generate_drift_report_slide_kwargs(suff_preds: dict) -> dict:
    content = [
        "Operational dataset ",
        SubText(f"{'is' if is_sufficient else 'is not'}", bold=True),
        f" sufficient for a desired model accuracy of {desired_accuracies[0]}",
    ]

    kwargs = {
        "title": "Sufficiency: Summary",
        "layout": DefaultGradientSlideLayouts.CONTENT_DEFAULT,
        "placeholder_fillings": [TextContent(lines=[Text(content=content)])],
        "additional_shapes": [
            Table(
                dataframe=generate_drift_report_table(suff_preds).round(4),
                fontsize=16,
                left=2.0,
                top=2.0,
                width=9.0,
                height=4.0,
            ),
        ],
    }
    return kwargs


def generate_slide_from_fig(fig, working_directory, fig_title):
    save_dir = os.path.join(working_directory, f"{fig_title}.png")
    fig.savefig(save_dir)
    kwargs = {
        "title": fig_title,
        "layout": DefaultGradientSlideLayouts.CONTENT_TITLE_ONLY,
        "additional_shapes": [
            Image(
                path=save_dir,
                placement=Placement.MANUAL,
                left=1.0,
                top=1.5,
                width=5.0,
                height=5.0,
            )
        ],
    }
    return kwargs

In [ ]:
from pathlib import Path

example_directory = Path.cwd() / "report_suff_example"
example_directory.mkdir(parents=True, exist_ok=True)

In [ ]:
# Generate and add to the slide deck
deck = SlideDeck(presentation_template_path=DEFAULT_GRADIENT_PRESENTATION_TEMPLATE_PATH)

deck.add_slide(**generate_drift_report_slide_kwargs(suff_preds))


for i, fig in enumerate(figs):
    fig_title = f"Sufficiency Plot {i}"

    deck.add_slide(**generate_slide_from_fig(fig, example_directory, fig_title))

In [ ]:
deck.save(
    output_directory=example_directory,
    name="report_suff_example",
)